In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


# Load data

In [2]:
data_path = 'C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/data/'
train = pd.read_csv(f'{data_path}train.csv')
test = pd.read_csv(f'{data_path}test.csv')
wp1 = pd.read_csv(f'{data_path}wp1.csv')
wp2 = pd.read_csv(f'{data_path}wp2.csv')
wp3 = pd.read_csv(f'{data_path}wp3.csv')
wp4 = pd.read_csv(f'{data_path}wp4.csv')
wp5 = pd.read_csv(f'{data_path}wp5.csv')
wp6 = pd.read_csv(f'{data_path}wp6.csv')


In [3]:
def get_df_name(df):
    name = [x for x in globals() if globals()[x] is df][0]
    return name

for df in [train,test,wp1,wp2,wp3,wp4,wp5,wp6]:
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d%H')
    print("NaN in Dataset: ",get_df_name(df), df.isnull().values.any())


NaN in Dataset:  train False
NaN in Dataset:  test False
NaN in Dataset:  wp1 True
NaN in Dataset:  wp2 True
NaN in Dataset:  wp3 True
NaN in Dataset:  wp4 True
NaN in Dataset:  wp5 True
NaN in Dataset:  wp6 True


In [4]:
train['train'] = 1
train = train.reset_index()
test = test.reset_index()
data_full = train.append(test)
data_full = data_full.sort_values(by='date')

In [5]:
def process_wp_file(wp):
    wp['contain_na'] = wp['u'].isna()
    wp['date_f'] = wp['date']+pd.to_timedelta(wp['hors'], unit='h')
    wp = wp.sort_values(by=['contain_na','date_f', 'date'], ascending=(True, True, False), na_position='last')
    wp.drop_duplicates(subset=['date_f'],keep='first',inplace=True)
    wp['date']=wp['date_f']
    wp=wp.drop(columns=['date_f','hors','contain_na'])
    return wp


In [6]:
wp1 = process_wp_file(wp1)
wp1 = pd.merge(data_full, wp1, on=['date'], how='left').drop(
    columns=['index','wp2', 'wp3', 'wp4', 'wp5', 'wp6'])
wp2 = process_wp_file(wp2)
wp2 = pd.merge(data_full, wp2, on=['date'], how='left').drop(
    columns=['index','wp1', 'wp3', 'wp4', 'wp5', 'wp6'])
wp3 = process_wp_file(wp3)
wp3 = pd.merge(data_full, wp3, on=['date'], how='left').drop(
    columns=['index', 'wp2', 'wp1', 'wp4', 'wp5', 'wp6'])
wp4 = process_wp_file(wp4)
wp4 = pd.merge(data_full, wp4, on=['date'], how='left').drop(
    columns=['index', 'wp2', 'wp3', 'wp1', 'wp5', 'wp6'])
wp5 = process_wp_file(wp5)
wp5 = pd.merge(data_full, wp5, on=['date'], how='left').drop(
    columns=['index', 'wp2', 'wp3', 'wp4', 'wp1', 'wp6'])
wp6 = process_wp_file(wp6)
wp6 = pd.merge(data_full, wp6, on=['date'], how='left').drop(
    columns=['index', 'wp2', 'wp3', 'wp4', 'wp5', 'wp1'])


In [7]:
# def mask_na(wp,option=1):
#     if option ==1:
#        wp[['u', 'v', 'ws', 'wd']]=wp[['u', 'v', 'ws', 'wd']].fillna(method='ffill', inplace=True)
#        return wp
#     elif option ==2:
#         wp.interpolate()
#         return wp
#     elif option ==3:
#         wp[['u', 'v', 'ws', 'wd']]=wp[['u', 'v', 'ws', 'wd']].fillna(method='bfill', inplace=True)
#         return wp
#     else:
#         from impyte import impyte
#         wp['date'] = wp['date'].astype(str)
#         imp = impyte.Impyter(wp)
#         wp= imp.impute(estimator='rf', multi_nans=True)
#         wp['date'] = pd.to_datetime(wp['date'], format='%Y%m%d%H')
#         return wp




In [8]:
fill_method='bfill'
columns = ['u', 'v', 'ws', 'wd']
wp1[columns] = wp1[columns].fillna(method=fill_method)
wp2[columns] = wp2[columns].fillna(method=fill_method)
wp3[columns] = wp3[columns].fillna(method=fill_method)
wp4[columns] = wp4[columns].fillna(method=fill_method)
wp5[columns] = wp5[columns].fillna(method=fill_method)
wp6[columns] = wp6[columns].fillna(method=fill_method)


In [9]:
save_address = 'C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/phase_2_Lucy/data/'
wp1.to_csv(save_address+'full_data_wp1.csv', index=False)
wp2.to_csv(save_address+'full_data_wp2.csv', index=False)
wp3.to_csv(save_address+'full_data_wp3.csv', index=False)
wp4.to_csv(save_address+'full_data_wp4.csv', index=False)
wp5.to_csv(save_address+'full_data_wp5.csv', index=False)
wp6.to_csv(save_address+'full_data_wp6.csv', index=False)


# Feature Engineering

In [10]:
save_address = 'C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/phase_2_Lucy/data/'
wp1= pd.read_csv(save_address+'full_data_wp1.csv')
wp2 = pd.read_csv(save_address+'full_data_wp2.csv')
wp3 = pd.read_csv(save_address+'full_data_wp3.csv')
wp4 = pd.read_csv(save_address+'full_data_wp4.csv')
wp5 = pd.read_csv(save_address+'full_data_wp5.csv')
wp6 = pd.read_csv(save_address+'full_data_wp6.csv')


In [11]:
#MInmax Scaler: This Scaler responds well if the standard deviation is small and when a distribution is not Gaussian. This Scaler is sensitive to outliers.
#Standar Scaler: The Standard Scaler assumes data is normally distributed within each feature and scales them such that the distribution centered around 0, with a standard deviation of 1.

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

def scale_features(df,scaler='StandardScaler'):
    if scaler == "MinMaxScaler":
        scaler = MinMaxScaler(feature_range =(0, 1))
    else:
        scaler = "StandardScaler"
        scaler = StandardScaler(feature_range =(0, 1))

    df[['u','v','ws','wd']] = scaler.fit_transform(df[['u','v','ws','wd']])
    
    return df

In [12]:
# from tsfresh.feature_extraction import extract_features, ComprehensiveFCParameters

# def add_ts_features(df):
#     extraction_settings = ComprehensiveFCParameters()

#     X = extract_features(df, column_id='id', column_sort='time',
#                      default_fc_parameters=extraction_settings,
#                      impute_function=impute)


In [13]:
def add_features(df):
    wv = df['ws']
    wd_rad = df['wd']*np.pi / 180

    # Calculate the wind x and y components.
    df['Wx'] = wv*np.cos(wd_rad)
    df['Wy'] = wv*np.sin(wd_rad)
    return df

def add_features2(df):
    #df['year']=pd.DatetimeIndex(df['date']).year
    #df['month']=pd.DatetimeIndex(df['date']).month
    # df['day']=pd.DatetimeIndex(df['date']).day
    # df['hour']=pd.DatetimeIndex(df['date']).hour
    # df['dayofweek_num']=pd.DatetimeIndex(df['date']).dayofweek  
    hour = 60*60
    day = 24*60*60
    month = (30)*day
    year = (365.2425)*day
    
    date_time = pd.DatetimeIndex(df['date'])
    timestamp_s = date_time.map(datetime.datetime.timestamp)

    #df['Hour sin'] = np.sin( timestamp_s* (2 * np.pi / hour))
    #df['Hour cos'] = np.cos( timestamp_s * (2 * np.pi / hour))
    df['Day sin'] = np.sin( timestamp_s* (2 * np.pi / day))
    df['Day cos'] = np.cos( timestamp_s * (2 * np.pi / day))
    #df['Month sin'] = np.sin( timestamp_s* (2 * np.pi / month))
    #df['Month cos'] = np.cos( timestamp_s * (2 * np.pi / month))
    df['Year sin'] = np.sin( timestamp_s * (2 * np.pi / year))
    df['Year cos'] = np.cos( timestamp_s * (2 * np.pi / year))
    return df
    
def add_features3(data):
    #data[['u','v','ws','wd']] = np.abs(data[['u','v','ws','wd']])
    data.index=data['date']
    data['lag_1_u'] = data['u'].shift(1)
    data['lag_1_v'] = data['v'].shift(1)
    data['lag_1_ws'] = data['ws'].shift(1)
    data['lag_1_wd'] = data['wd'].shift(1)
    data['lag_-1_u'] = data['u'].shift(-1)
    data['lag_-1_v'] = data['v'].shift(-1)
    data['lag_-1_ws'] = data['ws'].shift(-1)
    data['lag_-1_wd'] = data['wd'].shift(-1)

    # data['lag_1_Wx'] = data['Wx'].shift(1)
    # data['lag_1_Wy'] = data['Wy'].shift(1)
    # data['lag_-1_Wx'] = data['Wx'].shift(-1)
    # data['lag_-1_Wy'] = data['Wy'].shift(-1)
    #data['lag_1_wd'] = data['wd'].shift(1)
    #data['lag_-1_wd'] = data['wd'].shift(-1)

    data['expanding_mean_u'] = data['u'].expanding(1,center=True).mean()#,center=True
    data['expanding_mean_v'] = data['v'].expanding(1,center=True).mean()
    data['expanding_mean_ws'] = data['ws'].expanding(1,center=True).mean()
    data['expanding_mean_wd'] = data['wd'].expanding(1, center=True).mean()
    #data['expanding_mean_Wx'] = data['Wx'].expanding(1,center=True).mean()
    #data['expanding_mean_Wy'] = data['Wy'].expanding(1,center=True).mean()
    #data['expanding_mean_wd'] = data['wd'].expanding(2).mean()
    
    # data['rolling_mean_ws'] = data['ws'].rolling(window=2,closed='both').mean()
    # data['rolling_mean_wd'] = data['wd'].rolling(window=2,closed='both').mean()
    # data['rolling_mean_v'] = data['v'].rolling(window=2,closed='both').mean()
    # data['rolling_mean_u'] = data['u'].rolling(window=2,closed='both').mean()
    
    
    data['lag_2_u'] = data['u'].shift(2)
    data['lag_2_v'] = data['v'].shift(2)
    data['lag_2_ws'] = data['ws'].shift(2)
    data['lag_2_wd'] = data['wd'].shift(2)
    data['lag_-2_u'] = data['u'].shift(-2)
    data['lag_-2_v'] = data['v'].shift(-2)
    data['lag_-2_ws'] = data['ws'].shift(-2)
    data['lag_-2_wd'] = data['wd'].shift(-2)
    
    # data['rolling_mean_u_1'] = data['u'].rolling(2,closed='both').mean()
    # data['rolling_mean_v_1'] = data['v'].rolling(2,closed='both').mean()
    # data['rolling_mean_ws_1'] = data['ws'].rolling(2,closed='both').mean()
    # data['rolling_mean_wd_1'] = data['wd'].rolling(2,closed='both').mean()

    return data


In [14]:
def add_features4(df):
    """Adds Features to DataFrame and Takes Averages for Dates Before train_end_str"""    

    df['Is_day_part1'] = (pd.DatetimeIndex(df['date']).hour.isin([1,2,3,4,5,6])) *1
    df['Is_day_part2'] = (pd.DatetimeIndex(df['date']).hour.isin([7,8,9,10,11,12])) *1
    df['Is_day_part3'] = (pd.DatetimeIndex(df['date']).hour.isin([13,14,15,16,17,18])) *1
    df['Is_day_part4'] = (pd.DatetimeIndex(df['date']).hour.isin([19,20,21,22,23,24])) *1
    #df['Is_wknd'] = pd.DatetimeIndex(df['date']).dayofweek // 4 # Fri-Sun are 4-6, Monday is 0 so this is valid
    
    #df['Month'] = pd.DatetimeIndex(df['date']).month    

    return df

In [15]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()


def kMeansRes(scaled_data, k, alpha_k=0.02):
    inertia_o = np.square((scaled_data - scaled_data.mean(axis=0))).sum()
    # fit k-means
    kmeans = KMeans(n_clusters=k, random_state=0).fit(scaled_data)
    scaled_inertia = kmeans.inertia_ / inertia_o + alpha_k * k
    return scaled_inertia


def chooseBestKforKMeans(scaled_data, k_range):
    ans = []
    for k in k_range:
        scaled_inertia = kMeansRes(scaled_data, k)
        ans.append((k, scaled_inertia))
    results = pd.DataFrame(ans, columns=['k', 'Scaled Inertia']).set_index('k')
    best_k = results.idxmin()[0]
    return best_k, results

def i_want_graph(results):
    plt.figure(figsize=(7, 4))
    plt.plot(results, 'o')
    plt.title('Adjusted Inertia for each K')
    plt.xlabel('K')
    plt.ylabel('Adjusted Inertia')
    plt.xticks(range(1, 10, 1))
    plt.show()

def add_features5(df):

    df['year'] = pd.DatetimeIndex(df['date']).year
    k_range = range(1, 10)

    best_k, results = chooseBestKforKMeans(mms.fit_transform(
        df[df['year'].isin([2009, 2010])][['u', 'v', 'ws', 'wd']]), k_range)
    print('ok..my best k is ...',best_k)

    #i_want_graph(results)

    #print("hey, i override my best_k")
    #best_k = 4
    
    kmeans = KMeans(n_clusters=best_k, random_state=77).fit(mms.fit_transform(
        df[df['year'].isin([2009, 2010])][['u', 'v', 'ws', 'wd']]))
    array1 = kmeans.labels_
    array2=kmeans.predict(mms.fit_transform(df[df['year'].isin(
        [2011, 2012])][['u', 'v', 'ws', 'wd']]))
    
    array1=array1.tolist()
    array2=array2.tolist()
    
    predict = array1 + array2
    df['my_cluster'] = pd.Series(predict, index=df.index)
    df = df.drop(columns='year')
    return df



In [16]:
def add_features6(df):
    df['dff_u_1'] = df['u']-df['u'].shift(1)
    df['dff_v_1'] = df['v']-df['v'].shift(1)
    df['dff_ws_1'] = df['ws']-df['ws'].shift(1)
    df['dff_wd_1'] = df['wd']-df['wd'].shift(1)

    df['dff_u_-1'] = df['u']-df['u'].shift(-1)
    df['dff_v_-1'] = df['v']-df['v'].shift(-1)
    df['dff_ws_-1'] = df['ws']-df['ws'].shift(-1)
    df['dff_wd_-1'] = df['wd']-df['wd'].shift(-1)

    df['dff_u_mean'] = df['u']-df['u'].expanding(1, center=True).mean()
    df['dff_v_mean'] = df['v']-df['v'].expanding(1,center=True).mean()
    df['dff_ws_mean'] = df['ws']-df['ws'].expanding(1,center=True).mean()
    df['dff_wd_mean'] = df['wd']-df['wd'].expanding(1, center=True).mean()

    df['dff_u_1%'] = df['dff_u_1']/df['u']
    df['dff_v_1%'] = df['dff_v_1']/df['v']
    df['dff_ws_1%'] = df['dff_ws_1']/df['ws']
    df['dff_wd_1%'] = (df['wd']-df['wd'].shift(-1))/df['wd']
    df['dff_u_-1%'] = df['dff_u_-1']/df['u']
    df['dff_v_-1%'] = df['dff_v_-1']/df['v']
    df['dff_ws_-1%'] = df['dff_ws_-1']/df['ws']
    df['dff_wd_-1%'] = (df['wd']-df['wd'].shift(-1))/df['wd']

    df['dff_u_mean%'] = df['dff_u_mean']/df['u']
    df['dff_v_mean%'] = df['dff_v_mean']/df['v']
    df['dff_ws_mean%'] = df['dff_ws_mean']/df['ws']
    df['dff_wd_mean%'] = df['dff_wd_mean']/df['wd']



    return df


In [17]:
def add_features7(df):
    
    df['lag_3_u'] = df['u'].shift(3)
    df['lag_3_v'] = df['v'].shift(3)
    df['lag_3_ws'] = df['ws'].shift(3)
    df['lag_3_wd'] = df['wd'].shift(3)
    df['lag_-3_u'] = df['u'].shift(-3)
    df['lag_-3_v'] = df['v'].shift(-3)
    df['lag_-3_ws'] = df['ws'].shift(-3)
    df['lag_-3_wd'] = df['wd'].shift(-3)
    
    df['dff_u_2'] = df['u']-df['u'].shift(2)
    df['dff_v_2'] = df['v']-df['v'].shift(2)
    df['dff_ws_2'] = df['ws']-df['ws'].shift(2)
    df['dff_wd_2'] = df['wd']-df['wd'].shift(2)

    df['dff_u_-2'] = df['u']-df['u'].shift(-2)
    df['dff_v_-2'] = df['v']-df['v'].shift(-2)
    df['dff_ws_-2'] = df['ws']-df['ws'].shift(-2)
    df['dff_wd_-2'] = df['wd']-df['wd'].shift(-2)

    df['dff_u_mean_2'] = df['u']-df['u'].expanding(2, center=True).mean()
    df['dff_v_mean_2'] = df['v']-df['v'].expanding(2, center=True).mean()
    df['dff_ws_mean_2'] = df['ws']-df['ws'].expanding(2, center=True).mean()
    df['dff_wd_mean_2'] = df['wd']-df['wd'].expanding(2, center=True).mean()

    df['dff_u_2%'] = df['dff_u_2']/df['u']
    df['dff_v_2%'] = df['dff_v_2']/df['v']
    df['dff_ws_2%'] = df['dff_ws_2']/df['ws']
    df['dff_wd_2%'] = (df['wd']-df['wd'].shift(-2))/df['wd']
    df['dff_u_-2%'] = df['dff_u_-2']/df['u']
    df['dff_v_-2%'] = df['dff_v_-2']/df['v']
    df['dff_ws_-2%'] = df['dff_ws_-2']/df['ws']
    df['dff_wd_-2%'] = (df['wd']-df['wd'].shift(-2))/df['wd']

    df['dff_u_mean_2%'] = df['dff_u_mean_2']/df['u']
    df['dff_v_mean_2%'] = df['dff_v_mean_2']/df['v']
    df['dff_ws_mean_2%'] = df['dff_ws_mean_2']/df['ws']
    df['dff_wd_mean_2%'] = df['dff_wd_mean_2']/df['wd']

    return df


In [18]:
def add_features8(df):

    df['lag_4_u'] = df['u'].shift(4)
    df['lag_4_v'] = df['v'].shift(4)
    df['lag_4_ws'] = df['ws'].shift(4)
    df['lag_4_wd'] = df['wd'].shift(4)
    df['lag_-4_u'] = df['u'].shift(-4)
    df['lag_-4_v'] = df['v'].shift(-4)
    df['lag_-4_ws'] = df['ws'].shift(-4)
    df['lag_-4_wd'] = df['wd'].shift(-4)

    df['dff_u_3'] = df['u']-df['u'].shift(3)
    df['dff_v_3'] = df['v']-df['v'].shift(3)
    df['dff_ws_3'] = df['ws']-df['ws'].shift(3)
    df['dff_wd_3'] = df['wd']-df['wd'].shift(3)

    df['dff_u_-3'] = df['u']-df['u'].shift(-3)
    df['dff_v_-3'] = df['v']-df['v'].shift(-3)
    df['dff_ws_-3'] = df['ws']-df['ws'].shift(-3)
    df['dff_wd_-3'] = df['wd']-df['wd'].shift(-3)

    df['dff_u_mean_3'] = df['u']-df['u'].expanding(3, center=True).mean()
    df['dff_v_mean_3'] = df['v']-df['v'].expanding(3, center=True).mean()
    df['dff_ws_mean_3'] = df['ws']-df['ws'].expanding(3, center=True).mean()
    df['dff_wd_mean_3'] = df['wd']-df['wd'].expanding(3, center=True).mean()

    df['dff_u_3%'] = df['dff_u_3']/df['u']
    df['dff_v_3%'] = df['dff_v_3']/df['v']
    df['dff_ws_3%'] = df['dff_ws_3']/df['ws']
    df['dff_wd_3%'] = (df['wd']-df['wd'].shift(-3))/df['wd']
    df['dff_u_-3%'] = df['dff_u_-3']/df['u']
    df['dff_v_-3%'] = df['dff_v_-3']/df['v']
    df['dff_ws_-3%'] = df['dff_ws_-3']/df['ws']
    df['dff_wd_-3%'] = (df['wd']-df['wd'].shift(-3))/df['wd']

    df['dff_u_mean_3%'] = df['dff_u_mean_3']/df['u']
    df['dff_v_mean_3%'] = df['dff_v_mean_3']/df['v']
    df['dff_ws_mean_3%'] = df['dff_ws_mean_3']/df['ws']
    df['dff_wd_mean_3%'] = df['dff_wd_mean_3']/df['wd']


    return df


In [19]:
def add_features9(df):

    df['lag_5_u'] = df['u'].shift(5)
    df['lag_5_v'] = df['v'].shift(5)
    df['lag_5_ws'] = df['ws'].shift(5)
    df['lag_5_wd'] = df['wd'].shift(5)
    df['lag_-5_u'] = df['u'].shift(-5)
    df['lag_-5_v'] = df['v'].shift(-5)
    df['lag_-5_ws'] = df['ws'].shift(-5)
    df['lag_-5_wd'] = df['wd'].shift(-5)

    df['dff_u_4'] = df['u']-df['u'].shift(4)
    df['dff_v_4'] = df['v']-df['v'].shift(4)
    df['dff_ws_4'] = df['ws']-df['ws'].shift(4)
    df['dff_wd_4'] = df['wd']-df['wd'].shift(4)

    df['dff_u_-4'] = df['u']-df['u'].shift(-4)
    df['dff_v_-4'] = df['v']-df['v'].shift(-4)
    df['dff_ws_-4'] = df['ws']-df['ws'].shift(-4)
    df['dff_wd_-4'] = df['wd']-df['wd'].shift(-4)

    df['dff_u_mean_4'] = df['u']-df['u'].expanding(4, center=True).mean()
    df['dff_v_mean_4'] = df['v']-df['v'].expanding(4, center=True).mean()
    df['dff_ws_mean_4'] = df['ws']-df['ws'].expanding(4, center=True).mean()
    df['dff_wd_mean_4'] = df['wd']-df['wd'].expanding(4, center=True).mean()

    df['dff_u_4%'] = df['dff_u_4']/df['u']
    df['dff_v_4%'] = df['dff_v_4']/df['v']
    df['dff_ws_4%'] = df['dff_ws_4']/df['ws']
    df['dff_wd_4%'] = (df['wd']-df['wd'].shift(-4))/df['wd']
    df['dff_u_-4%'] = df['dff_u_-4']/df['u']
    df['dff_v_-4%'] = df['dff_v_-4']/df['v']
    df['dff_ws_-4%'] = df['dff_ws_-4']/df['ws']
    df['dff_wd_-4%'] = (df['wd']-df['wd'].shift(-4))/df['wd']

    df['dff_u_mean_4%'] = df['dff_u_mean_4']/df['u']
    df['dff_v_mean_4%'] = df['dff_v_mean_4']/df['v']
    df['dff_ws_mean_4%'] = df['dff_ws_mean_4']/df['ws']
    df['dff_wd_mean_4%'] = df['dff_wd_mean_4']/df['wd']

    df['lag_6_u'] = df['u'].shift(6)
    df['lag_6_v'] = df['v'].shift(6)
    df['lag_6_ws'] = df['ws'].shift(6)
    df['lag_6_wd'] = df['wd'].shift(6)
    df['lag_-6_u'] = df['u'].shift(-6)
    df['lag_-6_v'] = df['v'].shift(-6)
    df['lag_-6_ws'] = df['ws'].shift(-6)
    df['lag_-6_wd'] = df['wd'].shift(-6)

    df['dff_u_5'] = df['u']-df['u'].shift(5)
    df['dff_v_5'] = df['v']-df['v'].shift(5)
    df['dff_ws_5'] = df['ws']-df['ws'].shift(5)
    df['dff_wd_5'] = df['wd']-df['wd'].shift(5)

    df['dff_u_-5'] = df['u']-df['u'].shift(-5)
    df['dff_v_-5'] = df['v']-df['v'].shift(-5)
    df['dff_ws_-5'] = df['ws']-df['ws'].shift(-5)
    df['dff_wd_-5'] = df['wd']-df['wd'].shift(-5)

    df['dff_u_mean_5'] = df['u']-df['u'].expanding(5, center=True).mean()
    df['dff_v_mean_5'] = df['v']-df['v'].expanding(5, center=True).mean()
    df['dff_ws_mean_5'] = df['ws']-df['ws'].expanding(5, center=True).mean()
    df['dff_wd_mean_5'] = df['wd']-df['wd'].expanding(5, center=True).mean()

    df['dff_u_5%'] = df['dff_u_5']/df['u']
    df['dff_v_5%'] = df['dff_v_5']/df['v']
    df['dff_ws_5%'] = df['dff_ws_5']/df['ws']
    df['dff_wd_5%'] = (df['wd']-df['wd'].shift(-5))/df['wd']
    df['dff_u_-5%'] = df['dff_u_-5']/df['u']
    df['dff_v_-5%'] = df['dff_v_-5']/df['v']
    df['dff_ws_-5%'] = df['dff_ws_-5']/df['ws']
    df['dff_wd_-5%'] = (df['wd']-df['wd'].shift(-5))/df['wd']

    df['dff_u_mean_5%'] = df['dff_u_mean_5']/df['u']
    df['dff_v_mean_5%'] = df['dff_v_mean_5']/df['v']
    df['dff_ws_mean_5%'] = df['dff_ws_mean_5']/df['ws']
    df['dff_wd_mean_5%'] = df['dff_wd_mean_5']/df['wd']

    return df


In [20]:
def add_features10(df):

    df['lag_6_u'] = df['u'].shift(6)
    df['lag_6_v'] = df['v'].shift(6)
    df['lag_6_ws'] = df['ws'].shift(6)
    df['lag_6_wd'] = df['wd'].shift(6)
    df['lag_-6_u'] = df['u'].shift(-6)
    df['lag_-6_v'] = df['v'].shift(-6)
    df['lag_-6_ws'] = df['ws'].shift(-6)
    df['lag_-6_wd'] = df['wd'].shift(-6)

    df['dff_u_5'] = df['u']-df['u'].shift(5)
    df['dff_v_5'] = df['v']-df['v'].shift(5)
    df['dff_ws_5'] = df['ws']-df['ws'].shift(5)
    df['dff_wd_5'] = df['wd']-df['wd'].shift(5)

    df['dff_u_-5'] = df['u']-df['u'].shift(-5)
    df['dff_v_-5'] = df['v']-df['v'].shift(-5)
    df['dff_ws_-5'] = df['ws']-df['ws'].shift(-5)
    df['dff_wd_-5'] = df['wd']-df['wd'].shift(-5)

    df['dff_u_mean_5'] = df['u']-df['u'].expanding(5, center=True).mean()
    df['dff_v_mean_5'] = df['v']-df['v'].expanding(5, center=True).mean()
    df['dff_ws_mean_5'] = df['ws']-df['ws'].expanding(5, center=True).mean()
    df['dff_wd_mean_5'] = df['wd']-df['wd'].expanding(5, center=True).mean()

    df['dff_u_5%'] = df['dff_u_5']/df['u']
    df['dff_v_5%'] = df['dff_v_5']/df['v']
    df['dff_ws_5%'] = df['dff_ws_5']/df['ws']
    df['dff_wd_5%'] = (df['wd']-df['wd'].shift(-5))/df['wd']
    df['dff_u_-5%'] = df['dff_u_-5']/df['u']
    df['dff_v_-5%'] = df['dff_v_-5']/df['v']
    df['dff_ws_-5%'] = df['dff_ws_-5']/df['ws']
    df['dff_wd_-5%'] = (df['wd']-df['wd'].shift(-5))/df['wd']

    df['dff_u_mean_5%'] = df['dff_u_mean_5']/df['u']
    df['dff_v_mean_5%'] = df['dff_v_mean_5']/df['v']
    df['dff_ws_mean_5%'] = df['dff_ws_mean_5']/df['ws']
    df['dff_wd_mean_5%'] = df['dff_wd_mean_5']/df['wd']

    df['lag_7_u'] = df['u'].shift(7)
    df['lag_7_v'] = df['v'].shift(7)
    df['lag_7_ws'] = df['ws'].shift(7)
    df['lag_7_wd'] = df['wd'].shift(7)
    df['lag_-7_u'] = df['u'].shift(-7)
    df['lag_-7_v'] = df['v'].shift(-7)
    df['lag_-7_ws'] = df['ws'].shift(-7)
    df['lag_-7_wd'] = df['wd'].shift(-7)

    df['dff_u_6'] = df['u']-df['u'].shift(6)
    df['dff_v_6'] = df['v']-df['v'].shift(6)
    df['dff_ws_6'] = df['ws']-df['ws'].shift(6)
    df['dff_wd_6'] = df['wd']-df['wd'].shift(6)

    df['dff_u_-6'] = df['u']-df['u'].shift(-6)
    df['dff_v_-6'] = df['v']-df['v'].shift(-6)
    df['dff_ws_-6'] = df['ws']-df['ws'].shift(-6)
    df['dff_wd_-6'] = df['wd']-df['wd'].shift(-6)

    df['dff_u_mean_6'] = df['u']-df['u'].expanding(6, center=True).mean()
    df['dff_v_mean_6'] = df['v']-df['v'].expanding(6, center=True).mean()
    df['dff_ws_mean_6'] = df['ws']-df['ws'].expanding(6, center=True).mean()
    df['dff_wd_mean_6'] = df['wd']-df['wd'].expanding(6, center=True).mean()

    df['dff_u_6%'] = df['dff_u_6']/df['u']
    df['dff_v_6%'] = df['dff_v_6']/df['v']
    df['dff_ws_6%'] = df['dff_ws_6']/df['ws']
    df['dff_wd_6%'] = (df['wd']-df['wd'].shift(-6))/df['wd']
    df['dff_u_-6%'] = df['dff_u_-6']/df['u']
    df['dff_v_-6%'] = df['dff_v_-6']/df['v']
    df['dff_ws_-6%'] = df['dff_ws_-6']/df['ws']
    df['dff_wd_-6%'] = (df['wd']-df['wd'].shift(-6))/df['wd']

    df['dff_u_mean_6%'] = df['dff_u_mean_6']/df['u']
    df['dff_v_mean_6%'] = df['dff_v_mean_6']/df['v']
    df['dff_ws_mean_6%'] = df['dff_ws_mean_6']/df['ws']
    df['dff_wd_mean_6%'] = df['dff_wd_mean_6']/df['wd']

    return df


In [21]:
def add_features11(df):
    df['u_sin']=np.sin(df['u'])
    df['v_sin']=np.sin(df['v'])
    df['ws_sin']=np.sin(df['ws'])
    df['wd_sin']=np.sin(df['wd'])

    df['u_sin'] = np.cos(df['u'])
    df['v_sin'] = np.cos(df['v'])
    df['ws_sin'] = np.cos(df['ws'])
    df['wd_sin'] = np.cos(df['wd'])

    df['lag_8_u'] = df['u'].shift(8)
    df['lag_8_v'] = df['v'].shift(8)
    df['lag_8_ws'] = df['ws'].shift(8)
    df['lag_8_wd'] = df['wd'].shift(8)
    df['lag_-8_u'] = df['u'].shift(-8)
    df['lag_-8_v'] = df['v'].shift(-8)
    df['lag_-8_ws'] = df['ws'].shift(-8)
    df['lag_-8_wd'] = df['wd'].shift(-8)

    return df


In [22]:
# Sum_of_squared_distances = []
# wp1['year'] = pd.DatetimeIndex(wp1['date']).year
# K = range(1, 10)
# for num_clusters in K:
#  kmeans = KMeans(n_clusters=num_clusters)
#  kmeans.fit(wp1[wp1['year'].isin([2011, 2012])][['u', 'v', 'ws', 'wd']])
#  Sum_of_squared_distances.append(kmeans.inertia_)
# plt.plot(K, Sum_of_squared_distances)
# plt.xlabel('Values of K')
# plt.ylabel('Sum of squared distances/Inertia')
# plt.title('Elbow Method For Optimal k')
# plt.show()


In [23]:
def add_all_features(df):
    #df=scale_features(df)
    #df=add_features(df)
    df=add_features2(df)  
    df=add_features3(df)
    #df=add_features4(df)
    df=add_features5(df)
    df = add_features6(df)
    df = add_features7(df)
    df = add_features8(df)
    df = add_features9(df)
    df = add_features10(df)
    df = add_features11(df)
    return df


In [24]:
wp1=add_all_features(wp1)
wp2=add_all_features(wp2)
wp3=add_all_features(wp3)
wp4=add_all_features(wp4)
wp5=add_all_features(wp5)
wp6=add_all_features(wp6)



ok..my best k is ... 8
ok..my best k is ... 8
ok..my best k is ... 7
ok..my best k is ... 7
ok..my best k is ... 7
ok..my best k is ... 6


In [25]:
save_address = 'C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/phase_2_Lucy/feature/'
wp1.to_csv(save_address+'wp1.csv',index=False)
wp2.to_csv(save_address+'wp2.csv',index=False)
wp3.to_csv(save_address+'wp3.csv',index=False)
wp4.to_csv(save_address+'wp4.csv',index=False)
wp5.to_csv(save_address+'wp5.csv',index=False)
wp6.to_csv(save_address+'wp6.csv',index=False)
